In [83]:
import pandas as pd

In [84]:
matches = pd.read_csv("Predicting EPL Games/matches.csv", index_col=0)

In [85]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [86]:
matches["date"] = pd.to_datetime(matches["date"])

In [87]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [88]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [89]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [90]:
matches["day_code"] = matches["date"].dt.dayofweek

In [91]:
matches.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opp_code', 'hour',
       'day_code'],
      dtype='object')

In [92]:
from sklearn.preprocessing import LabelEncoder

In [93]:
formation_column = 'formation'
label_encoder = LabelEncoder()

In [94]:
matches['formation_encoded'] = label_encoder.fit_transform(matches[formation_column])

In [95]:
matches["target"] = (matches["result"] == "W").astype("int")

In [96]:
matches.dtypes

date                 datetime64[ns]
time                         object
comp                         object
round                        object
day                          object
venue                        object
result                       object
gf                          float64
ga                          float64
opponent                     object
xg                          float64
xga                         float64
poss                        float64
attendance                  float64
captain                      object
formation                    object
referee                      object
match report                 object
notes                       float64
sh                          float64
sot                         float64
dist                        float64
fk                          float64
pk                          float64
pkatt                       float64
season                        int64
team                         object
venue_code                  

In [97]:
from sklearn.ensemble import RandomForestClassifier

In [347]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

In [309]:
train = matches[matches["date"] < '2022-01-01']

In [310]:
test = matches[matches["date"] > '2022-01-01']

In [311]:
predictors = ["venue_code", "opp_code", "hour", "day_code", "formation_encoded"]

In [312]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=200, random_state=1)

In [313]:
preds = rf.predict(test[predictors])

In [314]:
from sklearn.metrics import accuracy_score

In [315]:
acc = accuracy_score(test["target"], preds)

In [316]:
acc

0.6195652173913043

In [317]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [318]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,143,29
1,76,28


In [319]:
from sklearn.metrics import precision_score

In [320]:
precision = precision_score(test["target"], preds)

In [321]:
grouped_matches = matches.groupby("team")

In [322]:
group = grouped_matches.get_group("Manchester City")

In [323]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,formation_encoded,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,0.0,0.0,2022,Manchester City,0,18,16,6,10,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,0.0,0.0,2022,Manchester City,1,15,15,5,10,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,2022,Manchester City,1,0,12,5,10,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,2022,Manchester City,0,10,15,5,10,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,0.0,0.0,2022,Manchester City,1,17,15,5,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2021-05-01,12:30,Premier League,Matchweek 34,Sat,Away,W,2.0,0.0,Crystal Palace,...,0.0,0.0,2021,Manchester City,0,6,12,5,12,1
56,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Chelsea,...,0.0,1.0,2021,Manchester City,1,5,17,5,4,0
57,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4.0,3.0,Newcastle Utd,...,0.0,0.0,2021,Manchester City,0,14,20,4,10,1
58,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2.0,3.0,Brighton,...,0.0,0.0,2021,Manchester City,0,3,19,1,10,0


In [324]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [325]:
matches.dtypes

date                 datetime64[ns]
time                         object
comp                         object
round                        object
day                          object
venue                        object
result                       object
gf                          float64
ga                          float64
opponent                     object
xg                          float64
xga                         float64
poss                        float64
attendance                  float64
captain                      object
formation                    object
referee                      object
match report                 object
notes                       float64
sh                          float64
sot                         float64
dist                        float64
fk                          float64
pk                          float64
pkatt                       float64
season                        int64
team                         object
venue_code                  

In [326]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "xg", "xga"]
new_cols = [f"{c}_rolling" for c in cols]

In [327]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling',
 'xg_rolling',
 'xga_rolling']

In [328]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling,xga_rolling
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,...,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333,1.433333,1.766667
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,...,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000,1.300000,1.733333
9,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,...,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000,1.366667,1.033333
11,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,...,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000,1.366667,0.633333
12,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,...,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333,1.400000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2022-03-14,20:00,Premier League,Matchweek 29,Mon,Away,D,0.0,0.0,Crystal Palace,...,2.333333,1.333333,19.000000,7.000000,15.366667,0.333333,0.333333,0.333333,2.100000,1.033333
44,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Away,W,2.0,0.0,Burnley,...,1.666667,0.333333,18.333333,7.333333,16.000000,0.333333,0.000000,0.000000,2.166667,0.700000
46,2022-04-10,16:30,Premier League,Matchweek 32,Sun,Home,D,2.0,2.0,Liverpool,...,2.000000,0.333333,20.000000,6.666667,16.133333,0.333333,0.000000,0.000000,2.233333,0.466667
49,2022-04-20,20:00,Premier League,Matchweek 30,Wed,Home,W,3.0,0.0,Brighton,...,1.333333,0.666667,15.666667,4.666667,16.700000,0.333333,0.000000,0.000000,2.033333,0.633333


In [329]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

C:\Users\anoth\AppData\Local\Temp\ipykernel_19020\4052147919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [330]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Premier League  Matchweek 29   
                        33 2022-03-18  20:00  Premier League  Matchweek 30   
                        34 2022-04-02  15:00  Premier League  Matchweek 31   
                        35 2022-04-08  20:00  Premier League  Matchweek 32   
                        36 2022-04-24  14:00  Premier League  Matchweek 34   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6   Sun  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7   Sat  Away      L  0.0  1.0  Manchester City  ...   
                        9   Sun  Home      L  0.0  1.0   Leicester City  ...   
                        11  Sun  Away      W  1.0  0.0   Manchester Utd  ...   
                        13  Sun  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 32  Sun  Away      W  1.0  0.0          Everton  ...   
                        33  Fri  Home      L  2.0  3.0     Leeds United  ...   
                        34  Sat  Home      W  2.0  1.0      Aston Villa  ...   
                        35  Fri  Away      L  0.0  1.0    Newcastle Utd  ...   
                        36  Sun  Away      L  0.0  1.0          Burnley  ...   

                            gf_rolling  ga_rolling  sh_rolling  sot_rolling  \
team                                                                          
Arsenal                 6     2.000000    1.333333    7.666667     3.666667   
                        7     1.666667    1.666667    5.333333     3.666667   
                        9     1.000000    1.666667    7.000000     3.666667   
                        11    0.666667    1.000000    9.666667     4.000000   
                        13    0.333333    0.666667    9.666667     2.666667   
...                                ...         ...         ...          ...   
Wolverhampton Wanderers 32    1.333333    1.000000   12.333333     3.666667   
                        33    1.666667    0.666667   12.333333     4.333333   
                        34    2.333333    1.000000   13.000000     5.333333   
                        35    1.666667    1.333333   13.000000     5.000000   
                        36    1.333333    1.666667   10.000000     4.666667   

                           dist_rolling fk_rolling pk_rolling pkatt_rolling  \
team                                                                          
Arsenal                 6     14.733333   0.666667   0.000000      0.000000   
                        7     15.766667   0.000000   0.000000      0.000000   
                        9     16.733333   0.666667   0.000000      0.000000   
                        11    16.033333   1.000000   0.000000      0.000000   
                        13    18.033333   1.000000   0.333333      0.333333   
...                                 ...        ...        ...           ...   
Wolverhampton Wanderers 32    19.300000   0.000000   0.000000      0.000000   
                        33    19.600000   0.000000   0.000000      0.000000   
                        34    19.833333   0.000000   0.000000      0.000000   
                        35    18.533333   0.000000   0.000000      0.000000   
 

In [331]:
matches_rolling = matches_rolling.droplevel('team')

In [332]:
matches_rolling.index = range(matches_rolling.shape[0])

In [333]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling,xga_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000,1.466667,1.766667
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000,1.000000,1.766667
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000,0.833333,1.633333
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000,0.733333,0.966667
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333,0.966667,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,Everton,...,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000,0.733333,1.866667
1313,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,Leeds United,...,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000,0.733333,1.333333
1314,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,Aston Villa,...,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000,0.833333,1.200000
1315,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,Newcastle Utd,...,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000,0.933333,1.600000


In [334]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [335]:
test.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opp_code', 'hour', 'day_code',
       'formation_encoded', 'target'],
      dtype='object')

In [336]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [337]:
precision

0.6521739130434783

In [338]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [339]:
combined

,actual,prediction,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,1,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [340]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "Westham United": "Westham",
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_values)

In [341]:
combined["new_team"] = combined["team"].map(mapping)

In [342]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [343]:
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,1,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
245,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
246,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
247,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd


In [344]:
merged[(merged["prediction_x"] ==1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()

actual_x
1    26
0    13
Name: count, dtype: int64

In [346]:
26/39

0.6666666666666666